<a href="https://colab.research.google.com/github/2303a51863/NLP/blob/main/ASSIGNMENT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install spacy

In [4]:
import pandas as pd
import re

In [9]:
df = pd.read_csv("bbc_news.csv")

In [10]:
print("First 3 rows of dataset:")
print(df.head(3))

First 3 rows of dataset:
                                               title  \
0  Ukraine: Angry Zelensky vows to punish Russian...   
1  War in Ukraine: Taking cover in a town under a...   
2         Ukraine war 'catastrophic for global food'   

                         pubDate  \
0  Mon, 07 Mar 2022 08:01:56 GMT   
1  Sun, 06 Mar 2022 22:49:58 GMT   
2  Mon, 07 Mar 2022 00:14:42 GMT   

                                               guid  \
0  https://www.bbc.co.uk/news/world-europe-60638042   
1  https://www.bbc.co.uk/news/world-europe-60641873   
2      https://www.bbc.co.uk/news/business-60623941   

                                                link  \
0  https://www.bbc.co.uk/news/world-europe-606380...   
1  https://www.bbc.co.uk/news/world-europe-606418...   
2  https://www.bbc.co.uk/news/business-60623941?a...   

                                         description  
0  The Ukrainian president says the country will ...  
1  Jeremy Bowen was on the frontline in Irpin, as

In [11]:
def check_noisy(text):
    return re.findall(r"[\n•]", (text))

In [12]:
df['noisy_chars'] = df['description'].astype(str).apply(check_noisy)
print("\nNoisy characters detected in each row:")
print(df[['description', 'noisy_chars']].head(3))


Noisy characters detected in each row:
                                         description noisy_chars
0  The Ukrainian president says the country will ...          []
1  Jeremy Bowen was on the frontline in Irpin, as...          []
2  One of the world's biggest fertiliser firms sa...          []


In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter

In [14]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [16]:
from nltk.tokenize import word_tokenize
# Function to clean, tokenize, stem, remove stop words
def preprocess_nltk(text):
    # Remove special characters and digits
    text = re.sub(r'[^A-Za-z\s]', '', str(text))
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]
    # Stem tokens
    stemmed = [ps.stem(t) for t in tokens]
    return stemmed

df['stemmed_tokens'] = df['description'].apply(preprocess_nltk)

In [17]:
from collections import Counter
# Get top 10 frequent stemmed words
all_stems = [stem for tokens in df['stemmed_tokens'] for stem in tokens]
stem_freq = Counter(all_stems).most_common(10)

print("\nTop 10 frequent stemmed words:")
print(stem_freq)


Top 10 frequent stemmed words:
[('say', 6397), ('bbc', 2408), ('england', 2338), ('year', 2251), ('world', 2170), ('peopl', 2021), ('first', 1909), ('new', 1894), ('win', 1747), ('uk', 1682)]


In [23]:
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample DataFrame (replace with your actual data)
# df = pd.DataFrame({'description': [...]})

def spacy_pipeline_pipe(texts):
    docs = nlp.pipe(texts)  # Faster than calling nlp() on each string
    result = []
    for doc in docs:
        tokens = [token.lemma_.lower() for token in doc
                  if token.is_alpha and token.pos_ in ["NOUN", "VERB"]]
        result.append(tokens)
    return result

# Apply optimized function
df['lemmas'] = spacy_pipeline_pipe(df['description'].astype(str))

In [22]:
# Get top 10 frequent lemmas
all_lemmas = [lemma for tokens in df['lemmas'] for lemma in tokens]
lemma_freq = Counter(all_lemmas).most_common(10)

print("\nTop 10 frequent lemmas:")
print(lemma_freq)


Top 10 frequent lemmas:
[('say', 7138), ('year', 2994), ('people', 2018), ('win', 1985), ('take', 1646), ('have', 1527), ('day', 1505), ('make', 1475), ('time', 1210), ('find', 1148)]
